In [11]:
from src.extract.google_trends import fetch_trends_data
from src.config.keywords import FINTECH_KEYWORDS
from src.config.countries import COUNTRIES 

In [12]:
df_test = fetch_trends_data("billetera digital","PE")
df_test.head()


,date,keyword,country_code,interest_score
0,2019-01-01,billetera digital,PE,0
1,2019-02-01,billetera digital,PE,0
2,2019-03-01,billetera digital,PE,0
3,2019-04-01,billetera digital,PE,0
4,2019-05-01,billetera digital,PE,0


In [13]:
df_test.describe()


,date,interest_score
count,73,73.000000
mean,2021-12-31 04:16:26,3.342466
min,2019-01-01 00:00:00,0.000000
25%,2020-07-01 00:00:00,0.000000
50%,2022-01-01 00:00:00,1.000000
75%,2023-07-01 00:00:00,2.000000
max,2025-01-01 00:00:00,100.000000
std,NaN,12.087895


In [17]:
df_test["interest_score"].max()

np.int64(100)

In [18]:
df_test = fetch_trends_data("Yape", "PE")
df_test.head()
df_test.describe()


,date,interest_score
count,73,73.000000
mean,2021-12-31 04:16:26,46.410959
min,2019-01-01 00:00:00,3.000000
25%,2020-07-01 00:00:00,25.000000
50%,2022-01-01 00:00:00,45.000000
75%,2023-07-01 00:00:00,73.000000
max,2025-01-01 00:00:00,100.000000
std,NaN,29.375838
